In [8]:
#| default_exp app 

In [9]:
# connect phoenix for tracing
import phoenix as px
phoenix_session = px.launch_app()

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [10]:
from openinference.instrumentation.dspy import DSPyInstrumentor
from opentelemetry import trace as trace_api
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

In [11]:
endpoint = "http://localhost:6006/v1/traces"
resource = Resource(attributes={})
tracer_provider = trace_sdk.TracerProvider(resource=resource)
span_otlp_exporter = OTLPSpanExporter(endpoint=endpoint)
tracer_provider.add_span_processor(SimpleSpanProcessor(span_exporter=span_otlp_exporter))

trace_api.set_tracer_provider(tracer_provider=tracer_provider)
DSPyInstrumentor().instrument()

In [12]:
#|export
import subprocess
import dspy
import logging

class GeneratePseudocode(dspy.Signature):
    """
    Transform a high-level task description into pseudocode.
    """
    task = dspy.InputField(desc="High-level description of the task.")
    pseudocode = dspy.OutputField(desc="Generated pseudocode for the task.")

class PseudocodeToCode(dspy.Signature):
    """
    Convert pseudocode into executable code.
    """
    task = dspy.InputField(desc="High-level description of the task.")
    pseudocode = dspy.InputField(desc="Pseudocode for the task.")
    code = dspy.OutputField(desc="Executable code generated from pseudocode.")

class CheckCodeCorrectness(dspy.Signature):
    """
    Determine if the generated code meets the task requirements.
    """
    code = dspy.InputField(desc="The code to check.")
    tests = dspy.InputField(desc="Tests to run against the code.")
    code_execution_output = dspy.InputField(desc="The output of running the code.")
    correctness = dspy.OutputField(desc="Bool")

class RefinePseudocode(dspy.Signature):
    """
    Refine pseudocode based on the feedback from code execution and test results.
    """
    code_output = dspy.InputField(desc="Output produced by executing the code.")
    test_output = dspy.InputField(desc="Output produced by running the tests.")
    errors = dspy.InputField(desc="Errors encountered during code execution and testing.")
    new_pseudocode = dspy.OutputField(desc="Refined pseudocode.")

class RefineCodeWithPreviousContext(dspy.Signature):
    """
    Refine code by incorporating feedback and previous attempts.
    """
    task = dspy.InputField(desc="High-level description of the task.")
    new_pseudocode = dspy.InputField(desc="Newly refined pseudocode.")
    previous_code_errors = dspy.InputField(desc="Previous code and errors encountered.")
    new_code = dspy.OutputField(desc="Newly refined code.")

class RefineTestsWithPreviousContext(dspy.Signature):
    """
    Refine tests by incorporating feedback and previous attempts.
    """
    task = dspy.InputField(desc="High-level description of the task.")
    new_code = dspy.InputField(desc="Newly refined code.")
    previous_tests_errors = dspy.InputField(desc="Previous tests and errors encountered.")
    new_tests = dspy.OutputField(desc="Newly refined tests.")

class GenerateTests(dspy.Signature):
    """
    Generate tests for the given code.
    IMPORTANT: ONLY OUTPUT THE TESTS AS RAW PYTHON TEXT AND NOTHING ELSE!!
    """
    task = dspy.InputField(desc="the task that the code has been generated for.")
    code = dspy.InputField(desc="the code to generate tests for.")
    tests = dspy.OutputField(desc="the tests to run.")


class IterativeCodeRefinement(dspy.Module):
    def __init__(self):
        super().__init__()
        
        self.logger = logging.getLogger('my_logger')
        self.logger.setLevel(logging.DEBUG)
        
        handler = logging.StreamHandler()
        handler.setLevel(logging.DEBUG)
        

        formatter = logging.Formatter('%(message)s')
        handler.setFormatter(formatter)

        self.logger.addHandler(handler)
        
        self.logger.info("Testing Log")
        self.generate_pseudocode = dspy.ChainOfThought(GeneratePseudocode)
        self.pseudocode_to_code = dspy.ChainOfThought(PseudocodeToCode)
        self.generate_code_tests = dspy.ChainOfThought(GenerateTests)
        self.check_code_correctness = dspy.ChainOfThought(CheckCodeCorrectness)
        self.refine_pseudocode = dspy.ChainOfThought(RefinePseudocode)
        self.refine_code_with_previous_context = dspy.ChainOfThought(RefineCodeWithPreviousContext)
        self.refine_tests_with_previous_context = dspy.ChainOfThought(RefineTestsWithPreviousContext)
    
    def execute_code(self, code):
        """
        Executes given Python code and captures the stdout, stderr, and return code.
        """
        result = subprocess.run(["python", "-c", code], capture_output=True, text=True)
        return result.stdout, result.stderr, result.returncode

    def forward(self, task):
        """
        Main method to iterate over code refinement based on task input.
        """
        # Initial pseudocode and code generation
        pseudocode = self.generate_pseudocode(task=task).pseudocode
        code = self.pseudocode_to_code(task=task, pseudocode=pseudocode).code
        tests = self.generate_code_tests(task=task, code=code).tests

        # Log initial outputs
        self.logger.info(f"Generated pseudocode: {pseudocode}")
        self.logger.info(f"Generated code: {code}")
        self.logger.info(f"Generated tests: {tests}")
        
        # Initial code execution
        stdout, stderr, returncode = self.execute_code(code + "\n\n" + tests)
        is_correct = bool(self.check_code_correctness(code=code, tests=tests, code_execution_output=stdout + stderr).correctness)

        # Iterative refinement loop
        while not is_correct:
            new_pseudocode = self.refine_pseudocode(code_output=stdout, test_output=stderr, errors=str(returncode)).new_pseudocode
            code = self.refine_code_with_previous_context(task=task, new_pseudocode=new_pseudocode, previous_code_errors=stderr).new_code
            tests = self.refine_tests_with_previous_context(task=task, new_code=code, previous_tests_errors=stderr).new_tests

            # Execute refined code
            stdout, stderr, returncode = self.execute_code(code + "\n\n" + tests)
            is_correct = bool(self.check_code_correctness(code=code, tests=tests, code_execution_output=stdout + stderr).correctness)

            # Log refinement iteration results
            self.logger.info(f"Refined pseudocode: {new_pseudocode}")
            self.logger.info(f"Refined code: {code}")
            self.logger.info(f"Refined tests: {tests}")
            self.logger.info(f"Execution output: {stdout}")
            self.logger.info(f"Is correct: {is_correct}")

        # Final return with refined code and tests
        return {"final_code": code, "final_tests": tests, "output": stdout, "errors": stderr}

In [13]:
#|export
llm = dspy.OllamaLocal("open-hermes-2-4_0", max_tokens=3000, model_type="chat")
dspy.settings.configure(lm=llm)

In [14]:
#|export
optimized_code = IterativeCodeRefinement()(task="Write a python function that adds 2 numbers and returns the result.")
print(optimized_code)

Testing Log
Testing Log
Generated pseudocode: ```python
def add_numbers(number1, number2):
    # Perform addition of two numbers
    sum = number1 + number2
    
    # Return the result
    return sum
```
Generated pseudocode: ```python
def add_numbers(number1, number2):
    # Perform addition of two numbers
    sum = number1 + number2
    
    # Return the result
    return sum
```
Generated code: ```python
def add_numbers(number1, number2):
    sum = number1 + number2
    return sum
```
Generated code: ```python
def add_numbers(number1, number2):
    sum = number1 + number2
    return sum
```
Generated tests: ```python
# Test 1: Adding two positive integers
assert add_numbers(2, 3) == 5

# Test 2: Adding two negative integers
assert add_numbers(-2, -3) == -5

# Test 3: Adding an integer and a float
assert add_numbers(1, 2.5) == 3.5

# Test 4: Adding two floats
assert add_numbers(1.5, 2.5) == 4.0

# Test 5: Adding zero to a number
assert add_numbers(5, 0) == 5

# Test 6: Adding a numb

{'final_code': '```python\ndef add_numbers(number1, number2):\n    sum = number1 + number2\n    return sum\n```', 'final_tests': '```python\n# Test 1: Adding two positive integers\nassert add_numbers(2, 3) == 5\n\n# Test 2: Adding two negative integers\nassert add_numbers(-2, -3) == -5\n\n# Test 3: Adding an integer and a float\nassert add_numbers(1, 2.5) == 3.5\n\n# Test 4: Adding two floats\nassert add_numbers(1.5, 2.5) == 4.0\n\n# Test 5: Adding zero to a number\nassert add_numbers(5, 0) == 5\n\n# Test 6: Adding a number to zero\nassert add_numbers(0, 5) == 5\n\n# Test 7: Adding two strings (should raise TypeError)\n# assert add_numbers("2", "3") # This should raise TypeError\n```', 'output': '', 'errors': '  File "<string>", line 1\n    ```python\n    ^\nSyntaxError: invalid syntax\n'}


In [ ]:
import nbdev
nbdev.export.nb_export('240403_dspy_codegen.ipynb', 'app')
print("export successful")
